In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'

In [27]:
os.environ['OPENAI_API_KEY'] = 'API Key Here'

In [11]:
import os, datetime, threading
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from plyer import notification
import pyttsx3
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

In [17]:
#Loading personal documents
loader = TextLoader("/Applications/PersonalBot/My_Personal.txt")
docs = loader.load()

#Spliting large documents into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(split_docs, embedding=embedding, persist_directory="chroma_db")

retriever = vectorstore.as_retriever()
llm = ChatOpenAI(model="gpt-4", temperature=0.3)
rag = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=None)

In [19]:
#Setting up Google Calendar API
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
creds = None
CREDENTIALS_PATH = "/Applications/PersonalBot/credentials.json"
if os.path.exists('token.pickle'):
    creds = pickle.load(open('token.pickle','rb'))
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.pickle','wb') as f: pickle.dump(creds, f)

In [21]:
calendar = build('calendar', 'v3', credentials=creds)

# this gets today's event 
def get_today_events():
    now = datetime.datetime.utcnow().isoformat() + 'Z'
    end = (datetime.datetime.utcnow() + datetime.timedelta(hours=24)).isoformat() + 'Z'
    events_result = calendar.events().list(
        calendarId='primary',
        timeMin=now,
        timeMax=end,
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    return events_result.get('items', [])

In [23]:
# Notification at Login
def notify_startup():
    advice = "It's lovely and sunny today—no umbrella needed." if datetime.datetime.now().hour < 18 else ""
    # Set empty history at start
    result = rag.invoke({
        "question": f"What is my schedule for {datetime.datetime.now().strftime('%A')}?",
        "chat_history": []
    })
    summary = result["answer"]
    evs = get_today_events()
    schedule = "\n".join([f"- {ev['summary']} at {ev['start'].get('dateTime', ev['start'].get('date'))}" for ev in evs]) or "Nothing scheduled"
    print(f"\n Good day, Pallavi!\n\n{advice}\n\n Here's your schedule today:\n{schedule}\n\n Knowledge Summary:\n{summary}\n")

In [11]:
import tkinter as tk
from tkinter import scrolledtext

In [25]:
# Tkinter GUI Chat Window
def run_gui():
    chat_history = [] 

    def send_message(event=None):
        user_input = input_box.get()
        if user_input.strip() == "":
            return
        chat_box.insert(tk.END, f"You: {user_input}\n")

        if user_input.lower() in ["exit", "quit"]:
            root.destroy()
            return

        if "remind me" in user_input.lower():
            import re
            m = re.search(r'(\d+)\s*min', user_input)
            if m:
                delay = int(m.group(1)) * 60
                def remind():
                    notification.notify(title="Reminder", message=user_input, timeout=5)
                threading.Timer(delay, remind).start()
                chat_box.insert(tk.END, f"Assistant: Got it! I’ll remind you in {m.group(1)} minutes.\n\n")
                input_box.delete(0, tk.END)
                return

        # Passing both question and history 
        try:
            resp = rag.run({
                "question": user_input,
                "chat_history": chat_history
            })
        except Exception as e:
            resp = f"[Error] {e}"

        chat_box.insert(tk.END, f"Assistant: {resp}\n\n")
        input_box.delete(0, tk.END)

        # Updating chat history with the latest interaction
        chat_history.append((user_input, resp))

    root = tk.Tk()
    root.title("Pallavi's Assistant")
    root.geometry("350x300+1200+700")
    root.attributes("-topmost", True)

    chat_box = scrolledtext.ScrolledText(root, wrap=tk.WORD, font=("Helvetica", 10))
    chat_box.pack(padx=10, pady=5, fill=tk.BOTH, expand=True)
    chat_box.insert(tk.END, "Hello Pallavi! I'm ready to assist you.\n\n")

    bottom_frame = tk.Frame(root)
    bottom_frame.pack(pady=5)

    input_box = tk.Entry(bottom_frame, width=30)
    input_box.pack(side=tk.LEFT, padx=(5, 5))
    input_box.bind('<Return>', send_message)

    send_button = tk.Button(bottom_frame, text="Send", command=send_message)
    send_button.pack(side=tk.RIGHT)

    root.mainloop()